In [9]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import mean
from pyspark.sql.functions import avg,col,count,when,sum,floor,expr

In [ ]:
sessionStart = SparkSession.builder.appName("Titanic calc").getOrCreate()
dataFrame = sessionStart.read.csv("Titanic-Dataset.csv",header =True,inferSchema=True)
sc = sessionStart.sparkContext
dataFrame.show(truncate=False)

In [6]:
#Calcalation of Mean of Tickets
print("Average for ticket fare")
new_columns = [col.replace('"', '') for col in dataFrame.columns]
dataFrame = dataFrame.toDF(*new_columns)
result_df = dataFrame.agg(mean("`Fare`").alias("Mean_ticket_fare"))
result_df.show()
dataFrame.describe().show()

Average for ticket fare
+----------------+
|Mean_ticket_fare|
+----------------+
|32.2042079685746|
+----------------+



24/03/09 20:57:13 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------+-----------------+-------------------+------------------+--------------------+------+------------------+------------------+-------------------+------------------+-----------------+-----+--------+
|summary|      PassengerId|           Survived|            Pclass|                Name|   Sex|               Age|             SibSp|              Parch|            Ticket|             Fare|Cabin|Embarked|
+-------+-----------------+-------------------+------------------+--------------------+------+------------------+------------------+-------------------+------------------+-----------------+-----+--------+
|  count|              891|                891|               891|                 891|   891|               714|               891|                891|               891|              891|  204|     889|
|   mean|            446.0| 0.3838383838383838| 2.308641975308642|                NULL|  NULL| 29.69911764705882|0.5230078563411896|0.38159371492704824|260318.54916792738| 32.20420

In [10]:
#Six point summary of age 
summary = dataFrame.groupBy("Survived").agg(expr("percentile_approx(Age, 0.0) as Min"),
                                      expr("percentile_approx(Age, 0.25) as Q1"),
                                      expr("percentile_approx(Age, 0.5) as Median"),
                                      expr("percentile_approx(Age, 0.75) as Q3"),
                                      expr("percentile_approx(Age, 1.0) as Max"),
                                      expr("percentile_approx(Age, 0.75) - percentile_approx(Age, 0.25) as IQR"))
summary.show()

+--------+----+----+------+----+----+----+
|Survived| Min|  Q1|Median|  Q3| Max| IQR|
+--------+----+----+------+----+----+----+
|       1|0.42|19.0|  28.0|36.0|80.0|17.0|
|       0| 1.0|21.0|  28.0|39.0|74.0|18.0|
+--------+----+----+------+----+----+----+



In [12]:
# Calculate survival rate based on siblings
survivalrate = dataFrame.groupBy("Sibsp") \
                  .agg((avg(when(col("Survived") == 1, 1).otherwise(0))).alias("Survival Rate"))
survivalrate.show()

+-----+-------------------+
|Sibsp|      Survival Rate|
+-----+-------------------+
|    1| 0.5358851674641149|
|    3|               0.25|
|    5|                0.0|
|    4|0.16666666666666666|
|    8|                0.0|
|    2| 0.4642857142857143|
|    0|0.34539473684210525|
+-----+-------------------+



In [13]:
#Survival rate based on siblings
survivalrate = dataFrame.groupBy("Sibsp") \
                  .agg((avg(when(col("Survived") == 1, 1).otherwise(0))).alias("Survival Rate"))
survivalrate.show()

+-----+-------------------+
|Sibsp|      Survival Rate|
+-----+-------------------+
|    1| 0.5358851674641149|
|    3|               0.25|
|    5|                0.0|
|    4|0.16666666666666666|
|    8|                0.0|
|    2| 0.4642857142857143|
|    0|0.34539473684210525|
+-----+-------------------+



In [14]:
print("Rate of Survival With siblings")
dataFrame.filter(dataFrame['SibSp'] > 0).agg(avg("Survived").alias("Rating-NoSiblings")).show()

Rate of Survival With siblings
+------------------+
| Rating-NoSiblings|
+------------------+
|0.4664310954063604|
+------------------+



In [15]:
#What is the probability of survival based on the gender
probability = dataFrame.groupBy("Sex") \
                   .agg((sum(when(col("Survived") == 1, 1).otherwise(0)) / count("*")).alias("Probability"))
probability.show()

+------+-------------------+
|   Sex|        Probability|
+------+-------------------+
|female| 0.7420382165605095|
|  male|0.18890814558058924|
+------+-------------------+



In [18]:
# Calculate Based on Age Groups

df = dataFrame.withColumn("Age_Group", floor(col("Age") / 10) * 10)

age_group_survival = df.groupBy("Age_Group").agg((sum(when(col("Survived") == 1, 1).otherwise(0)) / count("*")).alias("Survival Rate"))
gender_survival = df.groupBy("Age_Group", "Sex").agg((sum(when(col("Survived") == 1, 1).otherwise(0)) / count("*")).alias("Survival Rate"))

female_survival = gender_survival.filter(col("Sex") == "female").select("Age_Group", "Survival Rate").withColumnRenamed("Survival Rate", "Female Survival Rate")
male_survival = gender_survival.filter(col("Sex") == "male").select("Age_Group", "Survival Rate").withColumnRenamed("Survival Rate", "Male Survival Rate")

combined = female_survival.join(male_survival, "Age_Group", "outer").orderBy("Age_Group")
combined.show()

+---------+--------------------+-------------------+
|Age_Group|Female Survival Rate| Male Survival Rate|
+---------+--------------------+-------------------+
|     NULL|  0.6792452830188679|               NULL|
|     NULL|                NULL|0.12903225806451613|
|        0|  0.6333333333333333|            0.59375|
|       10|  0.7555555555555555|0.12280701754385964|
|       20|  0.7222222222222222|0.16891891891891891|
|       30|  0.8333333333333334|0.21495327102803738|
|       40|              0.6875|0.21052631578947367|
|       50|  0.8888888888888888|0.13333333333333333|
|       60|                 1.0|0.13333333333333333|
|       70|                NULL|                0.0|
|       80|                NULL|                1.0|
+---------+--------------------+-------------------+



In [19]:
#What is the average survival rate based on the Embarked City?

print("Embarked City Names")
dataFrame.select('Embarked').distinct().show()
print("Survival rates based on City ")
dataFrame.groupBy("Embarked").agg(avg("Survived").alias("Rate Of Survival")).show()

Embarked City Names
+--------+
|Embarked|
+--------+
|       Q|
|       C|
|       S|
|    NULL|
+--------+

Survival rates based on City 
+--------+-------------------+
|Embarked|   Rate Of Survival|
+--------+-------------------+
|       Q|0.38961038961038963|
|    NULL|                1.0|
|       C| 0.5535714285714286|
|       S|0.33695652173913043|
+--------+-------------------+



In [20]:
#A passenger from first class is more likely to sucummb then the passenger from 3rd Class

total_passengers_by_class = dataFrame.groupBy("Pclass").agg(count("*").alias("TotalPassengers"))

survivors_by_class = dataFrame.groupBy("Pclass").agg(sum(when(col("Survived") == 1, 1).otherwise(0)).alias("TotalSurvivors"))


survival_rate_by_class = survivors_by_class.join(total_passengers_by_class, "Pclass") \
                                           .withColumn("SurvivalRate", col("TotalSurvivors") / col("TotalPassengers"))
survival_rate_by_class.show()


+------+--------------+---------------+-------------------+
|Pclass|TotalSurvivors|TotalPassengers|       SurvivalRate|
+------+--------------+---------------+-------------------+
|     1|           136|            216| 0.6296296296296297|
|     3|           119|            491|0.24236252545824846|
|     2|            87|            184|0.47282608695652173|
+------+--------------+---------------+-------------------+



#Wrong as survival rate of first class is more than third class

In [21]:

#Group which has highest and lowest survival rate based on age,group,gender,class and boarding city

groups = ["Age", "Sex", "Pclass", "Embarked"]

survival_rates = dataFrame.groupBy(*groups) \
                   .agg(avg(when(col("Survived") == 1, 1).otherwise(0)).alias("SurvivalRate"))

highest_survival_group = survival_rates.orderBy(col("SurvivalRate").desc()).first()

lowest_survival_group = survival_rates.orderBy(col("SurvivalRate")).first()

print("Group with the highest survival rate:")
print(highest_survival_group)

print("\nGroup with the lowest survival rate:")
print(lowest_survival_group)

Group with the highest survival rate:
Row(Age=35.0, Sex='female', Pclass=2, Embarked='S', SurvivalRate=1.0)

Group with the lowest survival rate:
Row(Age=28.5, Sex='male', Pclass=3, Embarked='C', SurvivalRate=0.0)


In [24]:
sessionStart.stop()